# DELAY SUM - DREGON DATASET

A ideia desse Jupyter é usar o dataset Dregon para aplicar o algoritmo Delay Sum.
Segundo o dataset, http://dregon.inria.fr/datasets/dregon/, as coordenadas X, Y e Z em metros dos oito microfones são:

mic0 = [0.0420, 0.0615, -0.0410]

mic1 = [-0.0420, 0.0615, 0.0410]

mic2 = [-0.0615, 0.0420, -0.0410]

mic3 = [-0.0615, -0.0420, 0.0410]

mic4 = [-0.0420, -0.0615, -0.0410]

mic5 = [0.0420, -0.0615, 0.0410]

mic6 = [0.0615, -0.0420, -0.0410]

mic7 = [0.0615, 0.0420, 0.0410]

Para fazer uma configuração semelhante ao ReSpeaker 4-Mic-Array, vou utilizar apenas quatro microfones com o mesmo valor no eixo Z, mic1, mic3, mic5 e mic7.

In [1]:
import librosa
import matplotlib.pyplot as plt
from IPython.display import Audio
import math
import numpy as np

### Importando um áudio qualquer

Segundo o dataset, os áudios são nomeados como se segue:

XX_YY_ZZ.wav where XX denotes the relative azimuth, YY the relative elevation and ZZ the relative distance of the loudspeaker from the array.

O dataset foi gravado com 44100 Hz, e, como o librosa importa os áudios sempre com 22kHz, é preciso colocar sr=None como parâmetro adicional da função abaixo.

In [ ]:
originalAudio, freqAmostragem = librosa.load('/home/dimi/Downloads/DREGON_clean_recordings_speech/60_-30_1.2__1.wav', sr=None, mono=False)

In [ ]:
freqAmostragem

Ouvindo os microfones que serão utilizados (mesmo eixo Z):

In [ ]:
Audio(data=originalAudio[1], rate=freqAmostragem)

In [ ]:
Audio(data=originalAudio[3], rate=freqAmostragem)

In [ ]:
Audio(data=originalAudio[5], rate=freqAmostragem)

In [ ]:
Audio(data=originalAudio[7], rate=freqAmostragem)

### Função para verificar o delay entre dois sinais pela correlação

Essa função foi copiada e colada do algoritmo que já havia sido desenvolvido antes. É preciso mandar o argumento maxDelay, que é a quantidade de amostras contidas no maior delay possível entre dois microfones quaisquer.

In [2]:
def verificarDelay(sinalA, sinalB, maxDelay=15, verbose=False):
    
    # Verificando se os dois sinais tem o mesmo tamanho e se maxDelay é compatível
    if len(sinalA) != len(sinalB) or maxDelay >= len(sinalA)-1 :
        return False
    
    # Sei que o maior delay possível (em qtd de amostras) é maxDelay. Portanto, em cada iteração, 
    # a comparação entre os sinais para gerar a correlação se dará com arrays de tamanho 
    # len(sinal) - maxDelay
    tamanho = len(sinalA) - maxDelay
    
    # O delay será a qtd de amostras pra trás ou para frente que há
    # entre o sinal B em relação ao sinal A. Negativo significa que B está adiantado, 0 que não há
    # delay e positivo que está atrasado.
    delay = 0
    melhorCorrelacao = -1
    arrayCorrelacoes = []
    
    inicioA = 0
    fimA    = inicioA + tamanho
    inicioB = len(sinalB) - tamanho
    fimB    = inicioB + tamanho
    
    # Fazendo as iterações e calculando a correlação entre os sinais
    for i in range(-maxDelay, maxDelay+1):
        
        # Calculando a correlação da iteração atual
        corrAtual = np.corrcoef(sinalA[inicioA:fimA], sinalB[inicioB:fimB])[0][1]
        
        # Verificando se encontramos uma correlacao maior ainda para atualizar o delay
        if corrAtual > melhorCorrelacao:
            melhorCorrelacao = corrAtual
            delay = -i
        
        # Printando as iterações se verbose estiver ligado
        if verbose == True:
            arrayCorrelacoes.append(corrAtual)
            plt.title("Delay " + str(i))
            plt.plot(sinalA[inicioA:fimA])
            plt.plot(sinalB[inicioB:fimB])
            plt.show()
            print("Sinal A:", int(inicioA), "até", int(fimA))
            print("Sinal B:", int(inicioB), "até", int(fimB))
            print("Correlação Atual:", corrAtual)
            print("Melhor Correlação:", melhorCorrelacao, "(Delay " + str(delay) + ")\n\n\n")
            
        # Fazendo os indexes dos arrays da próxima iteração. De i = -maxDelay até 0, o Sinal A 
        # fica parado e o Sinal B vem vindo pra trás. A partir de i = 1 até maxDelay, o Sinal B
        # fica parado e o Sinal A vai embora.
        if i < 0:
            inicioB -= 1
            fimB     = inicioB + tamanho
        else:
            inicioA += 1
            fimA     = inicioA + tamanho
         
    # Se o verbose estiver ligado eu printo as correlações em função do delay tb
    if verbose == True:
        plt.title("Correlações em função dos delays")
        plt.plot(arrayCorrelacoes)
        plt.show()
    
    # Retornando o delay
    return -delay

A função definirDelayMaximo pode calcular a quantidade de amostras do maior delay possível entre dois microfones. Basta enviar o tamanho do lado do array quadrado.

In [10]:
def definirDelayMaximo(freqAmostragem, L, velocidadeSom=340):
    
    # O delay máximo (em qtd de amostras) dar-se-á na diagonal dos microfones
    diagonal     = math.sqrt(2 * (L**2))
    tempo        = diagonal / velocidadeSom
    qtdAmostras  = freqAmostragem * tempo
    
    # Adicionando mais 1 amostra pra garantir
    return int(qtdAmostras + 1)

### Verificando o funcionamento da função calculando o delay entre os microfones

A grande cagada é que cada microfone está numa aresta diferente do quadrado. Talvez seja necessário fazer uma troca de base, mas por enquanto da pra calcular o tamanho da aresta de boas.

In [5]:
coordenadasMic1 = [-0.0420, 0.0615]
coordenadasMic3 = [-0.0615, -0.0420]
coordenadasMic5 = [0.0420, -0.0615]
coordenadasMic7 = [0.0615, 0.0420]

vetorLado = [coordenadasMic1[0] - coordenadasMic3[0], coordenadasMic1[1] - coordenadasMic3[1]]
tamanhoLado = math.sqrt(vetorLado[0]**2 + vetorLado[1]**2)

delayMax = definirDelayMaximo(freqAmostragem, L=tamanhoLado)
print(delayMax)

NameError: name 'freqAmostragem' is not defined

Mostrando como a função faz o cálculo:

In [ ]:
verificarDelay(originalAudio[1], originalAudio[3], maxDelay=delayMax, verbose=True)

Calculando o delay entre os microfones tendo o mic1 como referência.

In [ ]:
delay13 = verificarDelay(originalAudio[1], originalAudio[3], maxDelay=delayMax)
delay15 = verificarDelay(originalAudio[1], originalAudio[5], maxDelay=delayMax)
delay17 = verificarDelay(originalAudio[1], originalAudio[7], maxDelay=delayMax)

print(delay13)
print(delay15)
print(delay17)

## Implementando o produto interno

http://www.labbookpages.co.uk/audio/beamforming/delayCalc.html

### Passando a origem pra cima do microfone 1

In [ ]:
coordenadasOriginaisMic1 = np.array([-0.0420, 0.0615, 0.0410])
coordenadasOriginaisMic3 = np.array([-0.0615, -0.0420, 0.0410])
coordenadasOriginaisMic5 = np.array([0.0420, -0.0615, 0.0410])
coordenadasOriginaisMic7 = np.array([0.0615, 0.0420, 0.0410])

coordenadasNovasMic1 = coordenadasOriginaisMic1 - coordenadasOriginaisMic1
coordenadasNovasMic3 = coordenadasOriginaisMic3 - coordenadasOriginaisMic1
coordenadasNovasMic5 = coordenadasOriginaisMic5 - coordenadasOriginaisMic1
coordenadasNovasMic7 = coordenadasOriginaisMic7 - coordenadasOriginaisMic1

#coordenadasNovasMic1[2] += 0.0410
#coordenadasNovasMic3[2] += 0.0410
#coordenadasNovasMic5[2] += 0.0410
#coordenadasNovasMic7[2] += 0.0410

In [ ]:
print(coordenadasNovasMic1)
print(coordenadasNovasMic3)
print(coordenadasNovasMic5)
print(coordenadasNovasMic7)

### Resolvendo o sistema com mínimos quadrados

https://riptutorial.com/numpy/example/16034/find-the-least-squares-solution-to-a-linear-system-with-np-linalg-lstsq

#### Exemplo de como usar a função

In [ ]:
# Definindo a equação matricial
A = np.array([[1,0,0,0],
              [0,1,0,0],
              [0,0,1,0],
              [0,0,0,1]])
b = np.array([1,2,3,4])

# Resolvendo x
x, residuals, rank, s = np.linalg.lstsq(A,b, rcond=None)

print(x)

#### Aplicando no problema dos microfones

A matriz dos coeficientes será as coordenadas dos microfones 3, 5 e 7 e a matriz b será os delays multiplicados pela velocidade do som.

|m3x m3y m3z| |wx| |vSom.delay13|

|m5x m5y m5z|.|wy|=|vSom.delay15|

|m7x m7y m7z| |wz| |vSom.delay17|

In [ ]:
A = np.array([coordenadasNovasMic3.tolist(),
              coordenadasNovasMic5.tolist(),
              coordenadasNovasMic7.tolist()])
b = np.array([delay13, delay15, delay17])

x, residuals, rank, s = np.linalg.lstsq(A,b, rcond=None)
print(x)

### Definindo algumas funções pra ajudar

In [17]:
def vetorUnitario(vetor):
    return vetor/np.linalg.norm(vetor)

def anguloEntreDoisVetores(v1, v2):
    v1_u = vetorUnitario(v1)
    v2_u = vetorUnitario(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

def radParaGraus(angRad):
    return (angRad*180)/math.pi

### Calculando os ângulos

Agora que já tenho o vetor da direção em que o som veio, vou passar a origem de novo para (0,0,0).

In [ ]:
x = x + coordenadasOriginaisMic1

O ângulo azimutal será o arco de tangente da divisão do eixo y do vetor direção pelo aixo x.

In [ ]:
radParaGraus(math.atan(x[1]/x[0]))

Já o ângulo de elevação será o ângulo entre os vetores direção e projeção de direção no plano XY.

In [ ]:
radParaGraus(anguloEntreDoisVetores(x, [x[0], x[1], 0]))

## Resumindo

In [31]:
nomeArquivo = "75_0_2.4__3.wav"

In [32]:
# ABRINDO O AUDIO
originalAudio, freqAmostragem = librosa.load('/home/dimi/Downloads/DREGON_clean_recordings_speech/'+nomeArquivo, sr=None, mono=False)

# GEOMETRIA DOS MICROFONES
coordenadasOriginaisMic1 = np.array([-0.0420, 0.0615, 0.0410])
coordenadasOriginaisMic3 = np.array([-0.0615, -0.0420, 0.0410])
coordenadasOriginaisMic5 = np.array([0.0420, -0.0615, 0.0410])
coordenadasOriginaisMic7 = np.array([0.0615, 0.0420, 0.0410])

coordenadasNovasMic1 = coordenadasOriginaisMic1 - coordenadasOriginaisMic1
coordenadasNovasMic3 = coordenadasOriginaisMic3 - coordenadasOriginaisMic1
coordenadasNovasMic5 = coordenadasOriginaisMic5 - coordenadasOriginaisMic1
coordenadasNovasMic7 = coordenadasOriginaisMic7 - coordenadasOriginaisMic1

# CALCULANDO O DELAY MAXIMO
vetorLado = [coordenadasOriginaisMic1[0] - coordenadasOriginaisMic3[0], coordenadasOriginaisMic1[1] - coordenadasOriginaisMic3[1]]
tamanhoLado = math.sqrt(vetorLado[0]**2 + vetorLado[1]**2)

delayMax = definirDelayMaximo(freqAmostragem, L=tamanhoLado)

# CALCULANDO O DELAY ENTRE OS MICROFONES
delay13 = verificarDelay(originalAudio[1], originalAudio[3], maxDelay=delayMax)
delay15 = verificarDelay(originalAudio[1], originalAudio[5], maxDelay=delayMax)
delay17 = verificarDelay(originalAudio[1], originalAudio[7], maxDelay=delayMax)

# RESOLVENDO O SISTEMA DO PRODUTO INTERNO E CHAMANDO O VETOR DA DIREÇÃO DE w
A = np.array([coordenadasNovasMic3.tolist(),
              coordenadasNovasMic5.tolist(),
              coordenadasNovasMic7.tolist()])
b = np.array([delay13, delay15, delay17])

w, residuals, rank, s = np.linalg.lstsq(A,b, rcond=None)

# VOLTANDO A ORIGEM PRO PONTO (0,0,0)
w = w + coordenadasOriginaisMic1

# CALCULANDO OS ANGULOS
azimutal = radParaGraus(math.atan(w[1]/w[0]))
elevacao = radParaGraus(anguloEntreDoisVetores(w, [w[0], w[1], 0]))

# PRINTANDO
print(w)
print(azimutal)
print(elevacao)

[2.45692238e+01 1.30690303e+02 4.10000000e-02]
79.35290654262631
0.01766530370024451
